# Gas Dosing Demo

I created a makefile to make interacting with the Demo easier: 

Start Docker containers with the SEC Nodes running on port 10800 (gas_dosing) and 10801 (reactorcell):

 ``` make sim```

Start Containers and additionally start frappy gui:

```make frappy``` 

In [1]:
import numpy as np
from bluesky import RunEngine
import bluesky.plan_stubs as bps
from bluesky.plan_stubs import sleep, rd
from bluesky.plans import scan, count

import databroker

from bluesky.preprocessors import run_decorator, SupplementalData
from bluesky import preprocessors as bpp
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from secop_ophyd.SECoPDevices import SECoPNodeDevice
from secop_ophyd.SECoPSignal import SECoPParamBackend
from bluesky.utils import ProgressBarManager

from tiled.client import from_uri
import NexusCreator

import time
from ophyd.status import Status


from bluesky.log import config_bluesky_logging

run = True

config_bluesky_logging(level='WARNING')
# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({},call_returns_result=True)
bec = BestEffortCallback()
RE.subscribe(bec)
RE.waiting_hook = ProgressBarManager()
RE.ignore_callback_exceptions = False



#Example of adding metadata to RE environment
investigation_id = "Nexus Demonstrator"

RE.md["investigation_id"] = investigation_id


client = from_uri("http://localhost:8000",api_key="secret")

def post_document(name,doc):
    client.post_document(name, doc)
    
RE.subscribe(post_document)

# Connect to Gas Dosing SEC Node and generate ophyd device tree
gas_dosing =  SECoPNodeDevice.create('localhost','10800',RE.loop)

# Connect to Reactor Cell SEC Node and generate ophyd device tree
reactor_cell =  SECoPNodeDevice.create('localhost','10801',RE.loop)

gas_dosing.class_from_instance()
reactor_cell.class_from_instance()


from genNodeClass import *

gas_dosing:Gas_dosing = gas_dosing
reactor_cell:Reactor_cell = reactor_cell

baseline = [gas_dosing.massflow_contr1.ramp]

sd = SupplementalData(baseline=baseline)

RE.preprocessors.append(sd)

gas_dosing ready
reactor_cell ready


In [3]:
from typing_extensions import Self


class SECOP2NEXUS():
    @classmethod
    async def create(cls,nodes:list[SECoPNodeDevice]) -> Self:
        converter = SECOP2NEXUS(nodes)

        converter.group_names = [(await node.equipment_id.get_value()).split('.')[0] for node in nodes]

        for node in nodes:
            for name,module in node.mod_devices.items():
                if hasattr(module,'meaning'):
                    meaning_arr = await module.meaning.get_value()
                    meaning:dict  = meaning_arr.item() 

                    new_element:tuple = (meaning['importance'],module) 

                    if converter.meaning_dict.__contains__(meaning['function']):

                        converter.meaning_dict[meaning['function']].append(new_element)

                    else:
                        converter.meaning_dict[meaning['function']] = [new_element] 

        ## main part of nx_struct
        await converter.gen_general_data_struct()     

        ## sample partof nx_struct
        await converter.gen_sample_data_struct()

        return converter

        



    def __init__(self,nodes:list[SECoPNodeDevice]) -> None:
        self.nodes:list[SECoPNodeDevice] = nodes
        self.meaning_dict:dict[str,list[tuple[float,SECoPReadableDevice]]] = {}
        self.group_names:list[str] = []

        self.header:str = """
entry:NXentry
\t@NX_class = "NXentry"
"""

        self.general_data_struct:str= ""
        self.sample_data_struct:str = ""

    
 

    async def gen_general_data_struct(self):
         ### Gerneral Data             
        self.general_data_struct = """
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype:NX_CHAR = "sample environment"
"""

        for node in self.nodes:
            node_text = await node.generate_nexus_struct()

            self.general_data_struct +=   "\t\t".join(node_text.splitlines(True))


    async def gen_sample_data_struct(self):
         ### Sample Data             
        self.sample_data_struct = """
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype:NX_CHAR = "sample"
"""

        for group in self.group_names:
            self.sample_data_struct += f"""
\t\t{group}:NXenvironment
\t\t\t@NX_class = "NXenvironment"
\t\t\tname:NX_CHAR = "TODO"
"""

        for meaning , mod_list in self.meaning_dict.items():
            sorted_by_importance = sorted(mod_list, key=lambda tup: tup[0])
            
            closest_to_sample = sorted_by_importance[0]

            closest_module:SECoPReadableDevice = closest_to_sample[1]


            sig_backend: SECoPParamBackend = closest_module.value._backend

            # check if vlaue is numeric (NXlog only supports mumerical values)
            if not sig_backend.is_number():
                continue 

            param_unit = f'"{sig_backend.get_unit()}"'

            unit_line = (
                f"\n\t\t\t\t@units = {param_unit}" if sig_backend.get_unit() is not None else ""
            )
            #TODO do this with links
            self.sample_data_struct += f"""
\t\t{meaning}:NXlog
\t\t\t@NX_class = NXlog
\t\t\tvalue:NX_FLOAT64 = {closest_module.value.name}{unit_line}
\t\t\t\t@type = "{sig_backend.describe_dict['SECoP_dtype']}"
\t\t\ttime:NX_FLOAT64 = {closest_module.value.name}-timestamp
\t\t\t\t@start = 0
\t\t\t\t@units = "s"
"""     
        


    def write_nx_struct(self,path:str):
        with open(path, "w") as file:
            file.write(self.get_nexus_struct())

    def get_nexus_struct(self):

        return self.header + self.sample_data_struct + self.general_data_struct

conv = await SECOP2NEXUS.create(nodes= [gas_dosing,reactor_cell])


conv.write_nx_struct("demo.nxstruct")



In [2]:
async def generate_nexus_struct(nodes:list[SECoPNodeDevice]):
    group_name = [(await node.equipment_id.get_value()).split('.')[0] for node in nodes]

    print(group_name)


    text = """
entry:NXentry
\t@NX_class = "NXentry"
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype: "sample"
\t\t\t@NX_class = "NX_CHAR" 
"""
    ### Only data with meaning  

    ### All Data (sample of type sample_env)

    text += """
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype:sample environment
\t\t\t@NX_class = "NX_CHAR"
"""

    for node in nodes:
        node_text = await node.generate_nexus_struct()

        text +=   "\t".join(node_text.splitlines(True))


    return text



generated_text = await generate_nexus_struct(nodes= [gas_dosing,reactor_cell])

with open("demo.nxstruct", "w") as file:
    file.write(generated_text)

['gas_dosing', 'reactor_cell']


In [2]:
import NexusCreator
run = client[-1]

variables = {}

for key in run.primary.data.keys():
    if key == 'time':
        continue
    variables[key] = np.asarray(run.primary.data.get(key)).tolist()
    variables[key+'-timestamp'] = np.asarray(run.primary.data.get('time')).tolist()

config_dict:dict = run.primary.metadata['descriptors'][0]['configuration']

#pprint(config_dict)

for key,value in config_dict.items():
    data_dict:dict = value['data']
    postfix = '-meaning'

    data_dict = {key: value for key, value in data_dict.items() if not key.endswith(postfix)}

    variables.update(data_dict)

    timestamp_dict:dict = value['timestamps']
    variables.update ({k+'-timestamp': v for k, v in timestamp_dict.items()})


pprint(variables)

# Preparing NexusCreator
flags = {
  'nxstruct': './demo.nxstruct',
  'input': variables,
  'output': './minimal.nxs'
  
}


NexusCreator.executeConversion(flags)

{'gas_dosing-backpressure_contr1-description': 'A simulated pressure '
                                               'controller ',
 'gas_dosing-backpressure_contr1-description-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-features': [],
 'gas_dosing-backpressure_contr1-features-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-group': 'pressure',
 'gas_dosing-backpressure_contr1-group-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-implementation': 'frappy_HZB.pressure_controller.PressureController',
 'gas_dosing-backpressure_contr1-implementation-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-interface_classes': ['Drivable'],
 'gas_dosing-backpressure_contr1-interface_classes-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-meaning-timestamp': 1725973265.5975358,
 'gas_dosing-backpressure_contr1-ramp': 500.0,
 'gas_dosing-backpressure_contr1-ramp-timestamp': 1725966492.964788,
 'gas_dosing-backpress



Conversion: OBJECT to Nexus

object ---> ./minimal.nxs



Reading NXstruct file:	./demo.nxstruct

Opening Nexus file:	./minimal.nxs

Creating entry...

Closing Nexus file



In [4]:
import NexusCreator

variableDictionary = {
  'a': 1,
  'b': 2,
  'c': 3,
  'variable': [1, 2, 3],
  'image': [[1, 2, 3], [4, 5,6]]
}

flags = {
  'nxstruct': './nexuscreator/examples/test2.nxstruct',
  'input': variableDictionary,
  'dictionary': False
}

NexusCreator.executeConversion(flags)
